# Fremtidsrettet data ved hjelp av API
### Oppgave 2 - Datainnsamling

I denne delen har gruppen valgt å benytte API-er, nærmere bestemt et API med JSON-format, som hovedmetode for datainnhenting. Værdataene hentes fra [YR sin API-tjeneste](https://api.met.no/weatherapi/locationforecast/2.0/documentation), ved hjelp av geografiske koordinater innhentet fra [Google Maps](https://www.google.com/maps). Dette gjør det mulig å hente ut spesifikke værdata for valgte lokasjoner. Datasettet viser værprognoser for utvalgte steder for den kommende uken, og inkluderer blant annet informasjon om temperatur, vindhastighet og fuktighet. Nedenfor har gruppen gitt koordinater til noen steder, som kan brukes for å hente ut værdata. Funksjonen _make_watherJSON(lat, lon)_ lager en JSON fil med hensyn på de gitte koordinater. 

In [15]:
import pandas as pd
import pandasql as sqldf
import requests

In [5]:
#breddegrad og lengdegrad er hentet fra Google Maps
locations = {
    "Stryn": (61.903901, 6.706899),
    "Longyearbyen": (78.225920, 15.625622),
    "Paris": (48.854289, 2.342042),
    "London": (51.501814, -0.140605),
    "Cape Town": (-33.922348, 18.424031),
    "New York": (40.710676, -74.006219),
    "Tokyo" : (35.681599, 139.767185)
}

def add_location(place, lat, lon):
    
    if place not in locations.keys():
        locations[place] = (lat, lon)
        print(f"{place} ble lagt til med koordinater ({lat}, {lon})")

    else:
        print(f"{place} finnes allerede i dictionary.")

    return locations

#print(locations)


In [8]:
#koden lager en JSON fil ut i fra koordinatene som blir puttet inn i linken 
def make_weatherJSON(place):

    if place not in locations.keys():
        raise Exception(f"{place} ikke funnet i {locations.keys()}")

    url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={locations[place][0]}&lon={locations[place][1]}"
    headers = {'User-Agent': 'my-weather-app/1.0'}  #nødvendig for Yr API
    response = requests.get(url, headers=headers)
    
    
    if response.status_code != 200:
        raise Exception(f"API call failed with status code {response.status_code}")
    
    data = response.json()

    if not data:
        raise Exception("Ingen data i JSON-objekt")
    
    return data

#make_weatherJSON("Løngyearbyen") #(feilmenlding)
#make_weatherJSON("Longyearbyen")


### Oppgave 3 - Databehandlling

Funksjonen _clean_weather_data(lat, lon)_ brukes til å identifisere hull i datasettene fra funksjonen _make_watherJSON(lat, lon)_. Dersom noen hull blir identifisert, fyller funksjonen hullene med gjennomsnittet fra de tidligere målingene. Funksjonen _check_NaN_counter(lat, lon)_ brukes som en kontroll, for å sjekke om det er noen hull igjen i datasettet. Dersom det ikke er noen hull, returneres 0. Til slutt blir daten presentert i funksjonen "get_temperatures_24(lat, lon)". Ved hjelp av list comprehensions, printes temperaturene for hver time i 24 timer.

In [10]:
#rydder manglende verdier med fillna()
def clean_weather_data(place):
    data = make_weatherJSON(place)
    timeseries = data["properties"]["timeseries"]

    lst = []
    for entry in timeseries:
        time = entry["time"]
        details = entry["data"]["instant"]["details"]
        lst.append({
            "Time": time,
            "Temperature": details.get("air_temperature"),
            "Rain": entry["data"].get("next_1_hours", {}).get("details", {}).get("precipitation_amount"),
            "Wind speed": details.get("wind_speed")
        })

    
    #lager JSON-filen om til en DataFrame (lister)
    df = pd.DataFrame(lst)


    #gjør om "Time" til datetime
    df["Time"] = pd.to_datetime(df["Time"], errors="coerce")


    #fyller inn evt. hull (NaN) med gjennomsnitt
    df.fillna(df.mean(numeric_only=True), inplace=True)
    
    return df


clean_weather_data("London")

,Time,Temperature,Rain,Wind speed
0,2025-04-03 14:00:00+00:00,17.6,0.0,4.6
1,2025-04-03 15:00:00+00:00,17.1,0.0,4.5
2,2025-04-03 16:00:00+00:00,16.1,0.0,4.5
3,2025-04-03 17:00:00+00:00,15.4,0.0,4.1
4,2025-04-03 18:00:00+00:00,14.5,0.0,3.5
...,...,...,...,...
87,2025-04-12 00:00:00+00:00,10.6,0.0,1.5
88,2025-04-12 06:00:00+00:00,7.2,0.0,0.7
89,2025-04-12 12:00:00+00:00,19.0,0.0,3.0
90,2025-04-12 18:00:00+00:00,16.8,0.0,3.5


In [12]:
#sjekker om det er noen hull i datasettet etterpå
def check_NaN_counter(place):
    data = clean_weather_data(place)
    missing_counts = data.isna().sum()
    
    print("Antall NaN-verdier per kolonne:")
    print(missing_counts)
    return missing_counts


check_NaN_counter("Tokyo")

Antall NaN-verdier per kolonne:
Time           0
Temperature    0
Rain           0
Wind speed     0
dtype: int64


Time           0
Temperature    0
Rain           0
Wind speed     0
dtype: int64

In [14]:
#henter ut temeraturer for de neste 24 timene
def get_temperatures_24(place):
    data = make_weatherJSON(place)
    timeseries = data["properties"]["timeseries"]
    
    temperatures = [
        (entry["time"], entry["data"]["instant"]["details"]["air_temperature"])
        for entry in timeseries[:24]
    ]
    
    return temperatures

#get_temperatures_24("Paris")